In [15]:
%pylab inline
import numpy as np 
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [16]:
from mlxtend.preprocessing import TransactionEncoder # allow us to take a list of baskets and encode the baskets in a form amenable for itemset mininin
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules # mine frequent itemsets and mine association rules from frequent itemsets

In [17]:
data_path = './data/groceries.csv'
transactions = []
with open(data_path, 'r') as fp:
    for line in fp:
        transaction = line.strip().split(',') # list of items in a basket
        transactions.append(transaction)

In [18]:
print(transactions[0:10])

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'], ['tropical fruit', 'yogurt', 'coffee'], ['whole milk'], ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'], ['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product'], ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'], ['rolls/buns'], ['other vegetables', 'UHT-milk', 'rolls/buns', 'bottled beer', 'liquor (appetizer)'], ['potted plants'], ['whole milk', 'cereals']]


In [19]:
te = TransactionEncoder()

In [20]:
te.fit(transactions) # configures the encoder to work with the data
te_ary = te.transform(transactions) # transform the data based on the configurations "learnt" from the dataset

In [21]:
#te_ary = te.fit(transactions).transform(transactions)

In [22]:
#te_ary = te.fit_transform(transactions)

In [23]:
te_ary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [30]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [33]:
df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [56]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True) # min_support between 0.005 and 0.1

In [57]:
# frequent_itemsets['itemset_size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # get number of items in set
# valid_itemsets = frequent_itemsets['itemset_size'] > 1 # get sets with more than one item
# frequent_itemsets = frequent_itemsets[valid_itemsets] # filter out rows with one item in item set

In [58]:
frequent_itemsets

,support,itemsets
0,0.033452,(UHT-milk)
1,0.017692,(baking powder)
2,0.052466,(beef)
3,0.033249,(berries)
4,0.026029,(beverages)
...,...,...
328,0.011998,"(whole milk, tropical fruit, root vegetables)"
329,0.014540,"(whole milk, root vegetables, yogurt)"
330,0.010473,"(whole milk, soda, yogurt)"
331,0.015150,"(whole milk, tropical fruit, yogurt)"


In [59]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.4)

In [60]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(beef),(other vegetables),0.052466,0.193493,0.019725,0.375969,1.943066,0.009574,1.292416
1,(other vegetables),(beef),0.193493,0.052466,0.019725,0.101944,1.943066,0.009574,1.055095
2,(beef),(rolls/buns),0.052466,0.183935,0.013625,0.259690,1.411858,0.003975,1.102329
3,(rolls/buns),(beef),0.183935,0.052466,0.013625,0.074074,1.411858,0.003975,1.023337
4,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628
...,...,...,...,...,...,...,...,...,...
451,"(whole milk, yogurt)",(whipped/sour cream),0.056024,0.071683,0.010880,0.194192,2.709053,0.006864,1.152033
452,"(whipped/sour cream, yogurt)",(whole milk),0.020742,0.255516,0.010880,0.524510,2.052747,0.005580,1.565719
453,(whole milk),"(whipped/sour cream, yogurt)",0.255516,0.020742,0.010880,0.042579,2.052747,0.005580,1.022807
454,(whipped/sour cream),"(whole milk, yogurt)",0.071683,0.056024,0.010880,0.151773,2.709053,0.006864,1.112881


In [29]:
# lift(A => B) = P(A and B)/(P(A)P(B))